In [27]:
import tensorflow as tf
from tensorflow.keras import preprocessing

import numpy as np

In [28]:
samples = ['너 오늘 이뻐 보인다',
          '나는 오늘 기분이 더러워',
          '끝내주는데, 좋은 일이 있나봐',
          '나 좋은 일이 생겼어',
          '아 오늘 진짜 짜증나',
          '환상적인데, 정말 좋은거 같아']

targets = [[1], [0], [1], [1], [0], [1]]

In [29]:
tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(samples)
sequences = tokenizer.texts_to_sequences(samples)
input_sequences = np.array(sequences)
labels = np.array(targets)
word_index = tokenizer.word_index

In [30]:
sequences

[[4, 1, 5, 6],
 [7, 1, 8, 9],
 [10, 2, 3, 11],
 [12, 2, 3, 13],
 [14, 1, 15, 16],
 [17, 18, 19, 20]]

In [31]:
input_sequences

array([[ 4,  1,  5,  6],
       [ 7,  1,  8,  9],
       [10,  2,  3, 11],
       [12,  2,  3, 13],
       [14,  1, 15, 16],
       [17, 18, 19, 20]])

In [32]:
labels

array([[1],
       [0],
       [1],
       [1],
       [0],
       [1]])

In [33]:
word_index, len(word_index)

({'같아': 20,
  '기분이': 8,
  '끝내주는데': 10,
  '나': 12,
  '나는': 7,
  '너': 4,
  '더러워': 9,
  '보인다': 6,
  '생겼어': 13,
  '아': 14,
  '오늘': 1,
  '이뻐': 5,
  '일이': 3,
  '있나봐': 11,
  '정말': 18,
  '좋은': 2,
  '좋은거': 19,
  '진짜': 15,
  '짜증나': 16,
  '환상적인데': 17},
 20)

In [34]:
from tensorflow.keras import backend
from tensorflow.keras import layers

In [35]:
batch_size = 2
num_epochs = 100
vocab_size = len(word_index) + 1
emb_size = 128
hidden_dimension = 256
output_dimension = 1

##Keras Sequential API##

모델을 구축할 수 있는 가장 간단한 형태의 API

스택 형태로 구현

In [36]:
model = tf.keras.Sequential([
  layers.Embedding(vocab_size, emb_size, input_length = 4),
  layers.Lambda(lambda x: tf.reduce_mean(x, axis = 1)),
  layers.Dense(hidden_dimension, activation='relu'),
  layers.Dense(output_dimension, activation='sigmoid')
])

model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [37]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 4, 128)            2688      
_________________________________________________________________
lambda_1 (Lambda)            (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               33024     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 257       
Total params: 35,969
Trainable params: 35,969
Non-trainable params: 0
_________________________________________________________________


In [38]:
model.fit(input_sequences, labels, epochs=num_epochs, batch_size=batch_size)

Epoch 1/100
3/3 [==============================] - 0s 4ms/step - loss: 0.6885 - accuracy: 0.8333
Epoch 2/100
3/3 [==============================] - 0s 2ms/step - loss: 0.6668 - accuracy: 1.0000
Epoch 3/100
3/3 [==============================] - 0s 3ms/step - loss: 0.6490 - accuracy: 1.0000
Epoch 4/100
3/3 [==============================] - 0s 2ms/step - loss: 0.6295 - accuracy: 1.0000
Epoch 5/100
3/3 [==============================] - 0s 2ms/step - loss: 0.6094 - accuracy: 1.0000
Epoch 6/100
3/3 [==============================] - 0s 2ms/step - loss: 0.5847 - accuracy: 1.0000
Epoch 7/100
3/3 [==============================] - 0s 2ms/step - loss: 0.5556 - accuracy: 1.0000
Epoch 8/100
3/3 [==============================] - 0s 2ms/step - loss: 0.5239 - accuracy: 1.0000
Epoch 9/100
3/3 [==============================] - 0s 2ms/step - loss: 0.4913 - accuracy: 1.0000
Epoch 10/100
3/3 [==============================] - 0s 1ms/step - loss: 0.4505 - accuracy: 1.0000
Epoch 11/100
3/3 [===========

##Keras Functional API##

In [39]:
inputs = layers.Input(shape = (4, ))
embed_output = layers.Embedding(vocab_size, emb_size)(inputs)
pooled_output = tf.reduce_mean(embed_output, axis=1)
hidden_layer = layers.Dense(hidden_dimension, activation='relu')(pooled_output)
outputs = layers.Dense(output_dimension, activation='sigmoid')(hidden_layer)

In [40]:
model = tf.keras.Model(inputs=inputs, outputs=outputs)

model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
              loss = 'binary_crossentropy',
              metrics = ['accuracy'])

In [41]:
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 4)]               0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 4, 128)            2688      
_________________________________________________________________
tf_op_layer_Mean (TensorFlow [(None, 128)]             0         
_________________________________________________________________
dense_4 (Dense)              (None, 256)               33024     
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 257       
Total params: 35,969
Trainable params: 35,969
Non-trainable params: 0
_________________________________________________________________


In [42]:
model.fit(input_sequences, labels, epochs=num_epochs, batch_size=batch_size)

Epoch 1/100
3/3 [==============================] - 0s 3ms/step - loss: 0.6892 - accuracy: 0.6667
Epoch 2/100
3/3 [==============================] - 0s 2ms/step - loss: 0.6707 - accuracy: 0.6667
Epoch 3/100
3/3 [==============================] - 0s 3ms/step - loss: 0.6554 - accuracy: 1.0000
Epoch 4/100
3/3 [==============================] - 0s 2ms/step - loss: 0.6398 - accuracy: 1.0000
Epoch 5/100
3/3 [==============================] - 0s 2ms/step - loss: 0.6209 - accuracy: 1.0000
Epoch 6/100
3/3 [==============================] - 0s 2ms/step - loss: 0.6002 - accuracy: 1.0000
Epoch 7/100
3/3 [==============================] - 0s 2ms/step - loss: 0.5743 - accuracy: 1.0000
Epoch 8/100
3/3 [==============================] - 0s 2ms/step - loss: 0.5461 - accuracy: 1.0000
Epoch 9/100
3/3 [==============================] - 0s 2ms/step - loss: 0.5098 - accuracy: 1.0000
Epoch 10/100
3/3 [==============================] - 0s 3ms/step - loss: 0.4712 - accuracy: 1.0000
Epoch 11/100
3/3 [===========

##Keras Custom Model(Subclassing)##

In [43]:
class CustomModel(tf.keras.Model):

  def __init__(self, vocab_size, embed_dimension, hidden_dimension, output_dimension):
    super(CustomModel, self).__init__(name='my_model')
    self.embedding = layers.Embedding(vocab_size, embed_dimension)
    self.dense_layer = layers.Dense(hidden_dimension, activation='relu')
    self.output_layer = layers.Dense(output_dimension, activation='sigmoid')

  def call(self, inputs):
    x = self.embedding(inputs)
    x = tf.reduce_mean(x, axis=1)
    x = self.dense_layer(x)
    x = self.output_layer(x)

    return x

In [44]:
model = CustomModel(vocab_size = vocab_size, 
                    embed_dimension = emb_size,
                    hidden_dimension = hidden_dimension,
                    output_dimension = output_dimension)

model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(input_sequences, labels, epochs=num_epochs, batch_size=batch_size)

Epoch 1/100
3/3 [==============================] - 0s 3ms/step - loss: 0.6947 - accuracy: 0.5000
Epoch 2/100
3/3 [==============================] - 0s 2ms/step - loss: 0.6766 - accuracy: 1.0000
Epoch 3/100
3/3 [==============================] - 0s 2ms/step - loss: 0.6617 - accuracy: 1.0000
Epoch 4/100
3/3 [==============================] - 0s 2ms/step - loss: 0.6453 - accuracy: 1.0000
Epoch 5/100
3/3 [==============================] - 0s 2ms/step - loss: 0.6276 - accuracy: 1.0000
Epoch 6/100
3/3 [==============================] - 0s 2ms/step - loss: 0.6066 - accuracy: 1.0000
Epoch 7/100
3/3 [==============================] - 0s 2ms/step - loss: 0.5833 - accuracy: 1.0000
Epoch 8/100
3/3 [==============================] - 0s 2ms/step - loss: 0.5557 - accuracy: 1.0000
Epoch 9/100
3/3 [==============================] - 0s 2ms/step - loss: 0.5232 - accuracy: 1.0000
Epoch 10/100
3/3 [==============================] - 0s 3ms/step - loss: 0.4864 - accuracy: 1.0000
Epoch 11/100
3/3 [===========

##Keras Custom Layer##

사용 편의를 위해 여러 레이어를 하나로 묶은 레이어를 구현할 때 사용

In [49]:
class CustomLayer(tf.keras.Model):
  def __init__(self, hidden_dimension, output_dimension, **kwargs):
    self.hidden_dimension = hidden_dimension
    self.output_dimension = output_dimension
    super(CustomLayer, self).__init__(**kwargs)

  def build(self, input_shape):
    self.dense_layer1 = layers.Dense(self.hidden_dimension, activation='relu')
    self.dense_layer2 = layers.Dense(self.output_dimension)

  def call(self, inputs):
    hidden_output = self.dense_layer1(inputs)
    return self.dense_layer2(hidden_output)

  def get_config(self):
    base_config = super(CustomLayer, self).get_config()
    base_config['hidden_dim'] = self.hidden_dimension
    base_config['output_dim'] = self.output_dim
    return base_config

  def from_config(cls, config):
    return cls(**config)

In [50]:
model = tf.keras.Sequential([
                             layers.Embedding(vocab_size, emb_size, input_length = 4),
                             layers.Lambda(lambda x: tf.reduce_mean(x, axis = 1)),
                             CustomLayer(hidden_dimension, output_dimension),
                             layers.Activation('sigmoid')])

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 4, 128)            2688      
_________________________________________________________________
lambda_4 (Lambda)            (None, 128)               0         
_________________________________________________________________
custom_layer (CustomLayer)   (None, 1)                 33281     
_________________________________________________________________
activation (Activation)      (None, 1)                 0         
Total params: 35,969
Trainable params: 35,969
Non-trainable params: 0
_________________________________________________________________


In [51]:
model.compile(optimizer=tf.keras.optimizers.Adam(0.001), loss='binary_crossentropy',
              metrics = ['accuracy'])

model.fit(input_sequences, labels, epochs=num_epochs, batch_size=batch_size)

Epoch 1/100
3/3 [==============================] - 0s 3ms/step - loss: 0.6934 - accuracy: 0.3333
Epoch 2/100
3/3 [==============================] - 0s 2ms/step - loss: 0.6748 - accuracy: 1.0000
Epoch 3/100
3/3 [==============================] - 0s 2ms/step - loss: 0.6580 - accuracy: 1.0000
Epoch 4/100
3/3 [==============================] - 0s 2ms/step - loss: 0.6434 - accuracy: 1.0000
Epoch 5/100
3/3 [==============================] - 0s 2ms/step - loss: 0.6265 - accuracy: 1.0000
Epoch 6/100
3/3 [==============================] - 0s 3ms/step - loss: 0.6067 - accuracy: 1.0000
Epoch 7/100
3/3 [==============================] - 0s 3ms/step - loss: 0.5863 - accuracy: 1.0000
Epoch 8/100
3/3 [==============================] - 0s 3ms/step - loss: 0.5604 - accuracy: 1.0000
Epoch 9/100
3/3 [==============================] - 0s 2ms/step - loss: 0.5309 - accuracy: 1.0000
Epoch 10/100
3/3 [==============================] - 0s 3ms/step - loss: 0.4995 - accuracy: 1.0000
Epoch 11/100
3/3 [===========